Install Required Packages

In [1]:
%pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple pureml-evaluate
%pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple pureml
%pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple pureml-policy
%pip install xlrd
%pip install lightgbm

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple
  Using cached https://test-files.pythonhosted.org/packages/02/cc/f703b69d72606d5df6ed6339b9d3d0503cb3f0aacd7e9b4087c440310cd8/pureml_evaluate-0.1.6.1-py3-none-any.whl (264 kB)
  Using cached daisy-1.0-py3-none-any.whl (45 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached reportlab-4.0.9-py3-none-any.whl (1.9 MB)
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached fairlearn-0.8.0-py3-none-any.whl (235 kB)
  Using cached pydantic-2.5.3-py3-none-any.whl (381 kB)
  Using cached scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.4 MB)
  Using cached plotly-5.18.0-py3-none-any.whl (15.6 MB)
  Using c

In [ ]:
# use 'pureml auth login' to login to your account
# pureml init -- Need to Run this in command line
# puremlconfig.yaml file will be created in the current directory
# Make sure to restart the kernel after running the above commands for the changes to take effect

In [1]:
import pureml

pureml.__version__

'0.4.5.1'

In [2]:
import pureml_evaluate 

pureml_evaluate.__version__

'0.1.6.1'

In [3]:
import pureml_policy

pureml_policy.__version__

'0.2.2.2'

Import all the Required Pacakges

In [4]:
# Import all the required files
import pureml
from pureml.decorators import load_data,transformer,dataset,model
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings

warnings.simplefilter("ignore")
rand_seed = 1234
np.random.seed(rand_seed)

In [5]:
@load_data()
def load_dataset():
    df = pd.read_excel(io='default of credit card clients.xls',header  = 1)

    return df

In [6]:
@transformer()
def remove_columns(df):
    return df.drop(['ID'],axis =1)

In [7]:
@transformer()
def rename_columns(df):
    return df.rename(columns={"PAY_0": "PAY_1","default payment next month":"default"})    

In [8]:
@transformer()
def dataset_imbalances(df):
    categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]

    for col_name in categorical_features:
        df[col_name] = df[col_name].astype("category")

    Y, A_str = df.loc[:, "default"], df.loc[:, "SEX"]
    X = pd.get_dummies(df.drop(columns=["default", "SEX"]))

    #A_str = A.map({1: "male", 2: "female"})

    A_str.value_counts(normalize=True)
    Y.value_counts(normalize=True)
    
    # Generate "Interest" column as a DataFrame_str
    interest_values = np.random.normal(loc=2 * Y, scale=A_str)
    interest_column = pd.DataFrame(interest_values, columns=["Interest"])

    # Concatenate "Interest" column with X DataFrame
    X = pd.concat([X, interest_column], axis=1)

    return {'X':X,'Y':Y,'A_str':A_str}


In [9]:
@transformer()
def resample_training_data(X_train, Y_train, A_train):
   
    negative_ids = Y_train[Y_train == 0].index
    positive_ids = Y_train[Y_train == 1].index
    balanced_ids = positive_ids.union(
        np.random.choice(a=negative_ids, size=len(positive_ids)))

    X_train = X_train.loc[balanced_ids, :]
    Y_train = Y_train.loc[balanced_ids]
    A_train = A_train.loc[balanced_ids]
    return  {"X_train": X_train, "Y_train":Y_train, "A_train": A_train}

In [10]:
from pureml.decorators import dataset

@dataset(label='credit_example_dataset_demo3',upload=True)
def create_dataset():
    df = load_dataset()
    df = remove_columns(df)
    df = rename_columns(df)
    data  = dataset_imbalances(df)
    X,Y,A_str = data['X'],data['Y'],data['A_str']
    X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(X, Y, A_str, test_size=0.35, stratify=Y)
    data = resample_training_data(X_train, y_train, A_train)
    X_train, y_train, A_train = data['X_train'],data['Y_train'],data['A_train']
    A_test = A_test.to_numpy()
    y_test = y_test.to_numpy()
    return {"x_train":X_train,"y_train":y_train,"x_test":X_test,"y_test":y_test,"sensitive_features" : A_test}
    

create_dataset()

Dataset details have not been found

ReadME file does not exist.
Creating an Empty ReadME file


Dataset has been created!

Dataset version:  v1

Dataset label:  credit_example_dataset_demo3:v1

Dataset and lineage have been registered!

{'x_train':        LIMIT_BAL  AGE  PAY_1  PAY_2  PAY_3  PAY_4  PAY_5  PAY_6  BILL_AMT1  \
 0          20000   24      2      2     -1     -1     -2     -2       3913   
 1         120000   26     -1      2      0      0      0      2       2682   
 3          50000   37      0      0      0      0      0      0      46990   
 4          50000   57     -1      0     -1      0      0      0       8617   
 8         140000   28      0      0      2      0      0      0      11285   
 ...          ...  ...    ...    ...    ...    ...    ...    ...        ...   
 29973     230000   35      1     -2     -2     -2     -2     -2          0   
 29976      40000   47      2      2      3      2      2      2      52358   
 29981      50000   44      1      2      2      2      0      0      38671   
 29981      50000   44      1      2      2      2      0      0      38671   
 29982      90000   36      0      0      0      0      0      0       7752   
 
        BILL_AMT2  ...  EDUCATION_2  ED

In [11]:
data = pureml.dataset.fetch('credit_example_dataset_demo3:v1')
x_test = data['x_test']
y_test = data['y_test']


In [12]:
from pureml.decorators import model
import pureml

@model(label='credit_example_model_demo3')
def create_model():
    data = pureml.dataset.fetch('credit_example_dataset_demo3:v1')
    x_train = data['x_train']
    y_train = data['y_train']
    lgb_params = {
    "metric": "auc",
    "learning_rate": 0.03,
    "num_leaves": 10,
    "max_depth": 3,
    "random_state": rand_seed,
    "n_jobs": 1}

    #pureml.log(params=lgb_params)
    estimator = Pipeline(
        steps=[
            ("preprocessing", StandardScaler()),
            ("classifier", lgb.LGBMClassifier(**lgb_params)),
        ]
    )

    estimator.fit(x_train, y_train)
    return estimator

create_model()

[LightGBM] [Info] Number of positive: 4313, number of negative: 4313
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002686 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3523
[LightGBM] [Info] Number of data points in the train set: 8626, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


Model details have not been found

ReadME file does not exist.
Creating an Empty ReadME file


Model has been created!

Model has been registered!

Model version:  v1

Model label:  credit_example_model_demo3:v1

No metrics are found in config
No params are found in config
No figures are found in config


Pipeline(steps=[('preprocessing', StandardScaler()),
                ('classifier',
                 LGBMClassifier(learning_rate=0.03, max_depth=3, metric='auc',
                                n_jobs=1, num_leaves=10, random_state=1234))])

In [13]:
import pureml


pureml.model.fetch(label='credit_example_model_demo3:v1')

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('classifier',
                 LGBMClassifier(learning_rate=0.03, max_depth=3, metric='auc',
                                n_jobs=1, num_leaves=10, random_state=1234))])

In [14]:
import pureml

pureml.predict.add(label ='credit_example_model_demo3:v1',paths={'predict':'predict.py'})

Predict Function has been registered!

In [15]:
import pureml

pureml.predict.fetch(label='credit_example_model_demo3:v1')

 predict file predict.py has been fetched

NYC 144

In [17]:
from pureml_policy import policy_eval
results = policy_eval.eval(framework_name  = "nyc144",
            label_model='credit_example_model_demo3:v1',
            label_dataset='credit_example_dataset_demo3:v1')

 Succesfully fetched the dataset

 Succesfully fetched the predictor

 Succesfully fetched the model

Succesfully fetched the framework details

Framework assigned to model

...

Could not update the assignments

Succesfully fetched the framework details

Report Generated

Use this URL to view Report: 
https://pub-cc8d623e43954c8fa75380adfd3bb762.r2.dev/reports/credit_example_model_demo3_report_8ao0dzpp.html

Reports Fetched

In [18]:
results

{'framework_details': {'name': 'nyc144'},
 'model_details': {'name': 'credit_example_model_demo3', 'version': 'v1'},
 'dataset_details': {'name': 'credit_example_dataset_demo3', 'version': 'v1'},
 'sensitive_columns': ['2', '1'],
 'policies': {'complete': [{'name': 'recall',
    'category': 'operational',
    'value': '0.525963149078727',
    'status': 'fail',
    'threshold': '0.8',
    'columns_sensitive': None},
   {'name': 'f1',
    'category': 'operational',
    'value': '0.6381033022861982',
    'status': 'fail',
    'threshold': '0.7',
    'columns_sensitive': None},
   {'name': 'accuracy',
    'category': 'operational',
    'value': '0.7964761904761904',
    'status': 'pass',
    'threshold': '0.7',
    'columns_sensitive': None},
   {'name': 'precision',
    'category': 'operational',
    'value': '0.8110202324580285',
    'status': 'pass',
    'threshold': '0.8',
    'columns_sensitive': None},
   {'name': 'disparate_impact',
    'category': 'fairness',
    'value': '0.842007

NIST

In [19]:
from pureml_policy import policy_eval
results = policy_eval.eval(framework_name  = "NIST",
            label_model='credit_example_model_demo3:v1',
            label_dataset='credit_example_dataset_demo3:v1')

 Succesfully fetched the dataset

 Succesfully fetched the predictor

 Succesfully fetched the model

Succesfully fetched the framework details

Framework assigned to model

...

Updated the Assignments

Succesfully fetched the framework details

Report Generated

Use this URL to view Report: 
https://pub-cc8d623e43954c8fa75380adfd3bb762.r2.dev/reports/credit_example_model_demo3_report_i2a2cilx.html

Reports Fetched

In [ ]:
results